# Using of OpenVINO custom layers

This guide, based on [openvino_pytorch_layers](https://github.com/dkurt/openvino_pytorch_layers) repository, shows how to support [TORCH.NN.FUNCTIONAL.GRID_SAMPLE](https://pytorch.org/docs/stable/generated/torch.nn.functional.grid_sample.html#torch-nn-functional-grid-sample) layer for Intel OpenVINO.

You can find more information about how to create and use OpenVINO Extensions to facilitate mapping of custom operations from framework model representation to OpenVINO representation [here](https://docs.openvino.ai/latest/openvino_docs_Extensibility_UG_Frontend_Extensions.html).

### 0. Install OpenVINO before running this Jupyter Notebook

You can use [this tutorial](https://docs.openvino.ai/latest/openvino_docs_install_guides_installing_openvino_from_archive_linux.html#doxid-openvino-docs-install-guides-installing-openvino-from-archive-linux) to install OpenVINO Runtime.

Configure OV environment with `source <INSTALL_DIR>/setupvars.sh`.

### 1. Clone repository with extensions examples:

In [ ]:
!git clone https://github.com/dkurt/openvino_pytorch_layers.git

### 2. Get ONNX sample model

In [ ]:
# Install dependencies

!pip install numpy torch

In [ ]:
# You'll get `model.onnx` file with model, `inp.npy`, `inp1.npy` files with input tensors 
# and `ref.npy`file with output for onnx model.

!python openvino_pytorch_layers/examples/grid_sample/export_model.py
!ls

### 3. Build extensions from sources:

In [ ]:
# You'll get `libuser_cpu_extension.so` file as result.

import os

os.chdir('./openvino_pytorch_layers/user_ie_extensions')
!mkdir build
os.chdir('./build')

!cmake .. && make

In [ ]:
os.chdir('../../..')
!ls

### 4. Infer OpenVINO model

In [ ]:
import numpy as np
from openvino.runtime import Core

# Load reference values
inp = np.load('inp.npy')
inp1 = np.load('inp1.npy')
ref_res = np.load('ref.npy')

# Create Core and register user extension
core = Core()
core.add_extension('openvino_pytorch_layers/user_ie_extensions/build/libuser_cpu_extension.so')

# You can get .xml and .bin OpenVINO model files with
# `mo --input_model model.onnx --extension /path/to/libuser_cpu_extension.so`
# or load model from .onnx file directly
model = core.read_model('model.onnx')
compiled_model = core.compile_model(model, 'CPU')

results = compiled_model.infer_new_request({'input': inp, 'input1': inp1})
predictions = next(iter(results.values()))

# compare ONNX and OV models results
diff = np.max(np.abs(ref_res-predictions))
print('Res diff: ' + str(diff))